In [1]:
!pip install llama-index
!pip install llama-parse

In [2]:
!pip install cryptography

In [3]:
import nest_asyncio

from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from IPython.display import Image, Markdown

from llama_parse import LlamaParse

from llama_index.core.node_parser import MarkdownElementNodeParser

In [4]:
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from dotenv import load_dotenv
# from llama_parse import LlamaParse
import nest_asyncio
import os
load_dotenv()
nest_asyncio.apply()

/Users/macintosh/TA-DOCUMENT/StudyZone/ComputerScience/Artificial Intelligence/FPT/MultimodalRAG-LlamaIndex-Guardrail/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.embed_model = embed_model



In [6]:

import llama_index.core
import os
api_key = os.getenv('GOOGLE_API_KEY')
llamaAPI_KEY = os.getenv('LlamaCloud_API_KEY')
llm = Gemini(model="models/gemini-1.5-flash",api_key=api_key)
Settings.llm = llm


In [7]:
document_path = "/Users/macintosh/TA-DOCUMENT/StudyZone/ComputerScience/Artificial Intelligence/FPT/MultimodalRAG-LlamaIndex-Guardrail/data/Dataset_employee_final.xlsx"

In [8]:
parser = LlamaParse(
    api_key=llamaAPI_KEY,
    result_type="markdown",
)

documents = parser.load_data(document_path)

Started parsing the file under job_id 0dcc061b-d3b9-482a-8d58-202f1aa6763b


In [9]:
len(documents)


6

In [10]:
print(documents[3].get_content())


# DNSite

|DNSite     |                   |          |            |          |            |        |                      |      |      |                                       |
|-----------|-------------------|----------|------------|----------|------------|--------|----------------------|------|------|---------------------------------------|
|Employee ID|Name               |DOB       |Personal ID |Role      |Account Name|Password|Credit Card Number    |Salary|Gender|API_KEY                                |
|E6484      |Jose Gross         |10/29/1995|397-887-7870|Engineer  |jgross1     |v#7S%ZZh|4805-5030-4126-8855883|111293|Male  |sk-28c2c49fac3b3c7931837d7610ea611de31a|
|E5150      |Susan Perkins      |08/20/1989|423-227-9002|Design    |sperkins1   |4)12YBtg|3752-4630-2663-428    |64778 |Male  |sk-ad6f0826a449663ab4e9ace3c9c621d5c1fe|
|E8193      |Jessica Barr       |01/06/1987|827-335-6144|Support   |jbarr1      |$90X7Sk7|3720-9981-2475-608    |34227 |Female|sk-5d6d472eef4ebb524aa8

In [11]:
node_parser = MarkdownElementNodeParser(llm=llm, num_workers=4)


In [12]:
nodes = node_parser.get_nodes_from_documents(documents[:10])


1it [00:00, 8701.88it/s]
1it [00:00, 14027.77it/s]
1it [00:00, 12826.62it/s]
1it [00:00, 10894.30it/s]
1it [00:00, 14716.86it/s]
1it [00:00, 11244.78it/s]


In [13]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)


In [14]:
len(nodes), len(base_nodes), len(objects)


(18, 6, 6)

In [15]:
print(objects[0].get_content())


Employee information including ID, name, contact details, role, and API key.,
with the following table title:
HCMSite,
with the following columns:
- Employee ID: None
- Name: None
- DOB: None
- Personal ID: None
- Role: None
- Account Name: None
- Password: None
- Credit Card Number: None
- Salary: None
- Gender: None
- API_KEY: None



SPLIT

In [22]:
from cryptography.fernet import Fernet

# Load the encryption key from .env
load_dotenv()
key = os.getenv("ENCRYPTION_KEY")
cipher_suite = Fernet(key)
print(key)


23bxGf2PMcEbOteqOlPwCkLt3sQmh_gVAwDk9qGQ1rM=


In [23]:
import json
from cryptography.fernet import Fernet
from dotenv import load_dotenv
import os

# Function to extract unique column titles from multiple documents
def extract_unique_column_titles(objects):
    column_names = set()  # Use a set to automatically handle duplicates

    for obj in objects:
        content = obj.get_content()

        if "with the following columns:" in content:
            lines = content.split("\n")
            for line in lines:
                if line.strip().startswith("-"):
                    column_name = line.split(":")[0].strip("-").strip()
                    column_names.add(column_name)  # Add to the set

    return list(column_names)  # Convert back to list for further use

# Function to encrypt a value
def encrypt_value(value):
    # Convert the value to a string before encoding
    value_str = str(value)
    return cipher_suite.encrypt(value_str.encode()).decode()


# Function to decrypt a value
def decrypt_value(encrypted_value):
    return cipher_suite.decrypt(encrypted_value.encode()).decode()


In [18]:
# Extract column titles from all the documents
column_titles = extract_unique_column_titles(objects)
print("Unique Column Titles:", column_titles)


Unique Column Titles: ['Password', 'Credit Card Number', 'Salary', 'Employee ID', 'Personal ID', 'Role', 'API_KEY', 'Account Name', 'Gender', 'Name', 'DOB']


In [20]:
from litellm import completion

def extract_table_data_with_titles(markdown_table, column_titles):
    # Convert column titles into a string for the prompt
    titles_str = ", ".join(column_titles)

    # Define the LLM prompt
    prompt = f"""
    You are a data extraction assistant. The following is a markdown-like table containing structured employee data.
    The table includes the following columns: {titles_str}.
    
    Your task is to:
    1. Extract all the data for the specified columns.
    2. Return the result in a clean JSON format, where each key is a column title, and its value is a list of corresponding values for that column.
    3. Do not include any markdown code block markers such as ```json or ``` in the output.
    4. Provide only the raw JSON content without any extra text or formatting.

    Markdown Table:
    {markdown_table}

    Output Example:
    {{
        "Employee ID": ["E6484", "E5150"],
        "Name": ["Jose Gross", "Susan Perkins"],
        "DOB": ["10/29/1995", "08/20/1989"],
        ...
    }}
    """

    # Send the request to LLM
    response = completion(
        model="gemini/gemini-1.5-flash",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Parse and return the JSON result
    return response['choices'][0]['message']['content']



In [24]:
# Dictionary to hold all encrypted data
all_encrypted_data = {}

# Loop through all documents and process each one
for i in range(len(documents)):
    # Extract data for the current document using LLM
    extracted_data_raw = extract_table_data_with_titles(documents[i].get_content(), column_titles)

    # Clean up the raw data (removing ```json and ```)
    extracted_data_raw = extracted_data_raw.lstrip('```json').rstrip('```').strip()

    # Parse the cleaned JSON data
    try:
        extracted_data = json.loads(extracted_data_raw)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for document {i+1}: {e}")
        continue

    # Encrypt all values in the extracted data
    for key, values in extracted_data.items():
        if key not in all_encrypted_data:
            all_encrypted_data[key] = []
        
        # Encrypt values and add them to the corresponding column in all_encrypted_data
        for value in values:
            encrypted_value = encrypt_value(value)
            all_encrypted_data[key].append(encrypted_value)

# Save all encrypted data to a JSON file
with open("encrypted_data.json", "w") as file:
    json.dump(all_encrypted_data, file, indent=4)

print("Encrypted Data Saved:")
print(json.dumps(all_encrypted_data, indent=4))


Encrypted Data Saved:
{
    "Employee ID": [
        "gAAAAABnPBVCb7kuZUSA3n_0NRou85Unqv645yJ9-llifdVLvHUx7rJXhzGTGEsP-Yqsfxq6GpOL2pmu7yTVXsym4RAnD8wbpQ==",
        "gAAAAABnPBVCkX8zVL7U4Q1yMHweifghHbeWfCTO7lOVHlagT4fskaI3FOF_9NZWjQ1qnuNvbZRqFgk8kGe9iI3TcmRZypS0Hg==",
        "gAAAAABnPBVCvC1iMjPJtLMk9k_bqM2rTFnBPs5BP12E7gOBeUnQdqSsHCPbWlcmBUp1iFrEps9NJe9OquwjRU3N-ulPj7AbhQ==",
        "gAAAAABnPBVCMn4pHz9bi2GW06oUCp0gI5PyJUrurvBfdZ_UOwaZ-Z4RHpmtLpwzFJflZN_vXtibkjMKPDIyghTGAd8s-zu9FA==",
        "gAAAAABnPBVCG9sNMfUeXcEeQwBB1GLObTOXENITjCDqpCOpLbTR3fzEd899ohBPb2RLmVl8QDjQTbKNsFy7ROfQP9LvKENFtg==",
        "gAAAAABnPBVCT6FODv5_r555gObv54NBrWrLz7AS4YCzbOblLpGGmQ8SbxCEcT6YtPoGlFAfUuEiCPXI6mxRwlj2wH47DBwtHg==",
        "gAAAAABnPBVCvTVRvVDvzb71a4WDzwdi64l9iYa5NPY2VKPTipT53GJKC0GJyATDN3w6nXnrqpEfnNKJFx0RPZYbM0qY35pVGg==",
        "gAAAAABnPBVCLnSl7gZmPs7ufbXMxiTvXYyFvlsUCv3Cs-8vuyWN2nwR5ZNGn244OzPYsIlXk9ei1Rsk_doK1Pkl_UctIGnQbw==",
        "gAAAAABnPBVCgM4XaRmRk8CDz6xv3dvrdXQ-khWohxUDnAIyIk

In [25]:
# Load encrypted data and decrypt it for testing purposes
with open("encrypted_data.json", "r") as file:
    loaded_encrypted_data = json.load(file)

# Decrypt all values in the JSON
decrypted_data = {}
for key, values in loaded_encrypted_data.items():
    decrypted_data[key] = [decrypt_value(value) for value in values]

print("\nDecrypted Data:")
print(json.dumps(decrypted_data, indent=4))



Decrypted Data:
{
    "Employee ID": [
        "E4762",
        "E7188",
        "E9532",
        "E1564",
        "E2945",
        "E5268",
        "E5177",
        "E9837",
        "E1711",
        "E6956",
        "E8775",
        "E1106",
        "E1754",
        "E2627",
        "E8659",
        "E1937",
        "E5509",
        "E5082",
        "E3732",
        "E4264",
        "E5859",
        "E8891",
        "E5373",
        "E6874",
        "E7744",
        "E4468",
        "E1705",
        "E3599",
        "E3222",
        "E8768",
        "E3897",
        "E1537",
        "E7216",
        "E7921",
        "E7036",
        "E3163",
        "E3292",
        "E9343",
        "E2207",
        "E7172",
        "E9994",
        "E8221",
        "E7021",
        "E4622",
        "E4560",
        "E9948",
        "E2641",
        "E5984",
        "E5353",
        "E9622",
        "E8250",
        "E5187",
        "E3659",
        "E3956",
        "E3251",
        "E5420",
        

In [77]:
import json
import logging

logging.getLogger("litellm").handlers.clear()

def log_result_to_file(result):
    """Write only the JSON data to the file directly without using logging."""
    with open("guardrail_log_file.txt", "a") as file:
        # Write the JSON data directly as a new line
        file.write(result + "\n")

In [29]:
import json

def global_rail(guardrail_type, activated, sanitized_output, is_valid, risk_score, threshold, response_text):
    """
    Standardizes the result format for all guardrail checks.
    """
    return {
        "guardrail_type": guardrail_type,
        "activated": activated,
        "guardrail_detail": {
            "sanitized_output": sanitized_output,
            "is_valid": is_valid,
            "risk_score/threshold": f"{risk_score}/{threshold}",
            "response_text": response_text
        }
    }


In [26]:
# Load encrypted data from the file
def load_encrypted_data(file_path="encrypted_data.json"):
    with open(file_path, "r") as file:
        return json.load(file)

In [155]:
def sanitize_output(prompt, matches):
    """Sanitize the output by replacing matched values with '[MASKED]'"""
    for match in matches:
        prompt = prompt.replace(match, "[MASKED]")
    return prompt

def check_data(prompt, role, threshold=2):
    """Check if the prompt contains any restricted data based on the user's role."""
    
    # Define the restricted columns for each role
    role_restrictions = {
        "employee": ["Salary", "Password", "Credit Card Number"],
        "manager": ["Salary", "Password"],
        "admin": []  # Admin can see everything, so no restrictions
    }

    # Extract the column titles (data types) that we need to check for in the prompt
    restricted_columns = role_restrictions.get(role, [])
    
    if not restricted_columns:
        # If no restrictions for the role (e.g., admin), we don't need to check anything
        return {
            "guardrail_type": "Data Check",
            "activated": False,
            "sanitized_output": prompt,
            "guardrail_detail": {
                "sanitized_output": prompt,
                "is_valid": True,
                "risk_score/threshold": "0/0",
                "response_text": prompt
            }
        }

    # Load the encrypted data
    encrypted_data = load_encrypted_data()

    matches = []
    sanitized_output = prompt

    # Iterate through the restricted columns and check against encrypted data
    for column in restricted_columns:
        if column in encrypted_data:
            encrypted_values = encrypted_data[column]

            # Decrypt the values and check if any of them are in the prompt
            decrypted_values = [decrypt_value(value) for value in encrypted_values]

            # Find matches between decrypted values and prompt
            for decrypted_value in decrypted_values:
                if decrypted_value in prompt:
                    matches.append(decrypted_value)
    
    # If any matches were found, sanitize the output
    if matches:
        sanitized_output = sanitize_output(prompt, matches)

    # Calculate the risk score (fixed for now, could be dynamic)
    is_valid = not bool(matches)  # If matches were found, it's invalid
    risk_score = len(matches)  # You can adjust this logic for risk score based on the number of matches

    # Standardize the result using the global_rail function
    result = global_rail(
        guardrail_type="Data Check",
        activated=bool(matches),  # Activated if matches found
        sanitized_output=sanitized_output,
        is_valid=is_valid,
        risk_score=risk_score,
        threshold=threshold,
        response_text=prompt
    )

    return result  # Returning the result dictionary

In [159]:
# Example usage
prompt_salary = "The employee's salary is 111293."
prompt_password = "Please verify the password: v#7S%ZZh."
role = "admin"  # Role can be "employee", "manager", "admin"

result_salary = check_data(prompt_salary, role)
result_password = check_data(prompt_password, role)

# Printing the result
print("Salary Check Result:", result_salary)
print("Password Check Result:", result_password)

Salary Check Result: {'guardrail_type': 'Data Check', 'activated': False, 'sanitized_output': "The employee's salary is 111293.", 'guardrail_detail': {'sanitized_output': "The employee's salary is 111293.", 'is_valid': True, 'risk_score/threshold': '0/0', 'response_text': "The employee's salary is 111293."}}
Password Check Result: {'guardrail_type': 'Data Check', 'activated': False, 'sanitized_output': 'Please verify the password: v#7S%ZZh.', 'guardrail_detail': {'sanitized_output': 'Please verify the password: v#7S%ZZh.', 'is_valid': True, 'risk_score/threshold': '0/0', 'response_text': 'Please verify the password: v#7S%ZZh.'}}


In [75]:
from llm_guard.input_scanners import TokenLimit
from llm_guard import scan_output
from litellm import completion
def check_token_limit(prompt, role="employee"):
    threshold = 4089
    response = completion(
        model="gemini/gemini-1.5-flash",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    response_text = response.choices[0].message.content
    
    scanner = TokenLimit(limit=threshold, encoding_name="cl100k_base")
    sanitized_output, is_valid, risk_score = scanner.scan(prompt)
    
    # Use the global rail to format the result
    result = global_rail(
        guardrail_type="Token limit",
        activated=not is_valid,
        sanitized_output=sanitized_output,
        is_valid=is_valid,
        risk_score=risk_score,
        threshold=threshold,
        response_text=response_text
    )
    
    # Convert result to JSON string for file writing if activated
    result_json = json.dumps(result)
    if result["activated"]:
        log_result_to_file(result_json)

    return result  # Returning JSON string for potential further use

In [78]:
from llm_guard.output_scanners import Bias
from llm_guard.output_scanners.bias import MatchType
from llm_guard import scan_output
from litellm import completion

def is_Bias(prompt, role="employee"):
    threshold = 0.7
    response = completion(
        model="gemini/gemini-1.5-flash",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    response_text = response.choices[0].message.content
    
    # Scan the output for bias
    bias_scanner = Bias(threshold=threshold, match_type=MatchType.FULL)
    sanitized_output, is_valid, risk_score = bias_scanner.scan(prompt, response_text)
    
    # Use the global rail to format the result
    result = global_rail(
        guardrail_type="Bias",
        activated=not is_valid,
        sanitized_output=sanitized_output,
        is_valid=is_valid,
        risk_score=risk_score,
        threshold=threshold,
        response_text=response_text
    )
    # Convert result to JSON string for file writing if activated
    result_json = json.dumps(result)
    if result["activated"]:
        log_result_to_file(result_json)

    return result  # Returning JSON string for potential further use

# # Example of how the function can be called:
# prompt = "Why should we give non-binary employees roles that involve public interactions? Doesn’t that affect our company image?"
# result = is_Bias(prompt,role="employee")
# print(result)


In [79]:
from presidio_analyzer import PatternRecognizer, Pattern, AnalyzerEngine, RecognizerRegistry
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
import json
import os   

#ENTITIES: https://microsoft.github.io/presidio/supported_entities/

# Step 1: Define Patterns for Credit Card, IP Address, OpenAI API Key, and Password

# Existing Credit Card Pattern
credit_card_pattern = Pattern(
    name="Credit Card Pattern", 
    regex=r"\b(?:\d{4}-){3}\d{4}\b|\b\d{16}\b",  # Regex for credit card numbers
    score=0.9  # Confidence score
)

# **New Password Pattern** (adjusted for a typical password format)
password_pattern = Pattern(
    name="Password Pattern",
    regex=r"Password\sis\s([A-Za-z\d!\"#$%&'()*+,-./:;<=>?@[\\\]^_`{|}~]+)",
    score=0.9  # Confidence score
)

# Step 2: Create PatternRecognizers for Credit Card, IP Address, OpenAI API Key, and Password

# Existing Credit Card Recognizer
credit_card_recognizer = PatternRecognizer(
    supported_entity="CREDIT_CARD",  
    patterns=[credit_card_pattern]
)


# **New Password Recognizer**
password_recognizer = PatternRecognizer(
    supported_entity="PASSWORD",
    patterns=[password_pattern]
)

# Step 3: Initialize Recognizer Registry and Add Custom Recognizers

registry = RecognizerRegistry()
registry.load_predefined_recognizers()  # Load default recognizers

# Add custom recognizers
registry.add_recognizer(credit_card_recognizer)    # Add credit card recognizer
registry.add_recognizer(password_recognizer)        # Add password recognizer

# Step 4: Initialize the Analyzer and Anonymizer with the Updated Registry

analyzer = AnalyzerEngine(registry=registry,log_decision_process=False)
anonymizer = AnonymizerEngine()

# Step 5: Define Roles and Corresponding Entities Access
ROLE_ENTITY_MAPPING = {
    "admin": ["IP_ADDRESS"], 
    "manager": ["API_KEY"],  # Manager cannot access PASSWORD and API_KEY
    "employee": ["US_SSN"]
}

def detect_and_anonymize_pii(text, role="employee"):
    """Detect and anonymize PII, with role-based access control."""
    # Get the allowed entities for the given role
    allowed_entities = ROLE_ENTITY_MAPPING.get(role.lower())

    if allowed_entities is None:
        raise ValueError(f"Role '{role}' is not recognized. Allowed roles are: 'admin', 'manager', 'employee'.")

    # Analyze the input text for PII entities, only for those allowed by the role
    analyzer_results = analyzer.analyze(
        text=text, 
        language="en", 
        entities=allowed_entities
    )

    # Check if any PII entities were detected
    pii_detected = len(analyzer_results) > 0
    pii_types = list(set(result.entity_type for result in analyzer_results))

    # Anonymize the detected PII in the text
    anonymized_results = anonymizer.anonymize(
        text=text,
        analyzer_results=analyzer_results,
        operators={"DEFAULT": OperatorConfig("replace", {"new_value": "[MASKED]"})}
    )

    # Convert anonymized result to JSON for structured output
    anonymized_json = json.loads(anonymized_results.to_json())
    # Return structured response
    return {
        "result": anonymized_results.text,
        "guardrail_type": "PII",
        "activated": pii_detected,
        "details": {
            "pii_types": pii_types,
            "anonymized_details": anonymized_json,
            "analyzer_results":  analyzer_results
        }
    }

# Example input text with password and other PII
response_text = "David's credit card number is 2120-5550-5555-1211"

# Detect and anonymize PII in the query engine response for different roles
role = "employee"  # Example role: manager
pii_result = detect_and_anonymize_pii(response_text, role)

# Print the results
print("\nAnonymized Result:")
print(pii_result["details"])
print("\nWas PII detected?")
print("Yes" if pii_result["activated"] else "No")

# Uncomment the following lines to see detailed PII analysis
# print("\nDetailed PII Analysis:")
# print(json.dumps(pii_result["details"], indent=2))



Anonymized Result:
{'pii_types': [], 'anonymized_details': {'text': "David's credit card number is 2120-5550-5555-1211", 'items': []}, 'analyzer_results': []}

Was PII detected?
No


In [80]:
from llm_guard.input_scanners import Toxicity
from llm_guard.input_scanners.toxicity import MatchType
from litellm import completion

def check_toxic(prompt, role="employee"):
    # Interact with the LLM to generate a response
    print(f"Prompt: {prompt}")
    
    # Generate the response using the LLM (Gemini-1.5-flash)
    response = completion(
        model="gemini/gemini-1.5-flash",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    response_text = response.choices[0].message.content
    
    # Define the threshold and scan for toxicity
    threshold = 0.5
    toxic_scanner = Toxicity(threshold=threshold, match_type=MatchType.FULL)
    sanitized_output, is_valid, risk_score = toxic_scanner.scan(prompt)
    
    # Use the global rail function to format the result
    result = global_rail(
        guardrail_type="Toxicity",
        activated=not is_valid,  # Activated if toxicity is detected
        sanitized_output=sanitized_output,
        is_valid=is_valid,
        risk_score=risk_score,
        threshold=threshold,
        response_text=response_text
    )
    
    # Convert result to JSON string for file writing if activated
    result_json = json.dumps(result)
    if result["activated"]:
        log_result_to_file(result_json)

    return result  # Returning JSON string for potential further use

# Example usage
prompt = "You are idiot"
result = check_toxic(prompt, role="employee")
print(result)


Prompt: You are idiot
2024-11-19 12:23:47 [debug    ] Initialized classification model device=device(type='mps') model=Model(path='unitary/unbiased-toxic-roberta', subfolder='', revision='36295dd80b422dc49f40052021430dae76241adc', onnx_path='ProtectAI/unbiased-toxic-roberta-onnx', onnx_revision='34480fa958f6657ad835c345808475755b6974a7', onnx_subfolder='', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='mps'), 'padding': 'max_length', 'top_k': None, 'function_to_apply': 'sigmoid', 'return_token_type_ids': False, 'max_length': 512, 'truncation': True}, tokenizer_kwargs={})
2024-11-19 12:23:51 [warning  ] Detected toxicity in the text  results=[{'label': 'toxicity', 'score': 0.9967994689941406}, {'label': 'insult', 'score': 0.9958475232124329}]
{'guardrail_type': 'Toxicity', 'activated': True, 'guardrail_detail': {'sanitized_output': 'You are idiot', 'is_valid': False, 'risk_score/threshold': '1.0/0.5', 'response_text': "I understand that y

In [81]:
def InputScanner(query, listOfScanners, role="employee"):
    """
    Runs all scanners on the query and returns:
    - True if any scanner detects a threat.
    - A list of results from scanners that returned True.
    """
    detected = False  # Track if any scanner detects a threat
    triggered_scanners = []  # Store results from triggered scanners

    # Run each scanner on the query
    for scanner in listOfScanners:
        if scanner.__name__ == "detect_and_anonymize_pii":
            result = scanner(query, role="employee")  # Pass role to the PII scanner
        else:
            result = scanner(query,role="employee")  # Execute the scanner function
        
        if result["activated"]:  # Check if the scanner found a threat (activated=True)
            detected = True  # Set detected to True if any scanner triggers
            triggered_scanners.append(result)  # Track which scanner triggered

    return detected, triggered_scanners

#example
scanners = [detect_and_anonymize_pii]
query = "Credit card number: ID 9390-5600-6784-7740, my IP_Address is 192.168.0.1"
detected, triggered_scanners = InputScanner(query, scanners,role="employee")
print(triggered_scanners)

[]


In [82]:
def OutputScanner(response, query, context, listOfScanners,role="employee"):
    """
    Runs all scanners on the response and returns:
    - True if any scanner detects a threat.
    - A list of results from scanners that returned True.
    """
    detected = False  # Track if any scanner detects a threat
    triggered_scanners = []  # Store results from triggered scanners

    # Run each scanner on the response
    for scanner in listOfScanners:
        # Check if scanner is `evaluate_rag_response` (which needs query & context)
        if scanner.__name__ == "evaluate_rag_response":
            result = scanner(response, query, context,role=role)  # Execute with query & context
        else:
            result = scanner(response,role)  # Default scanner execution
        
        # print(f"Debug Output Scanner Result: {result}")

        if result["activated"]:  # Check if the scanner was triggered
            detected = True
            triggered_scanners.append(result)  # Track which scanner triggered

    return detected, triggered_scanners


In [83]:
from llama_index.core.query_engine import CustomQueryEngine, SimpleMultiModalQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.core.schema import ImageNode, NodeWithScore, MetadataMode
from llama_index.core.prompts import PromptTemplate
from llama_index.core.base.response.schema import Response
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from typing import Optional


QA_PROMPT_TMPL = """\
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query. 


Query: {query_str}
Answer: """

QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)


from typing import List, Callable, Optional
from pydantic import Field
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.prompts import PromptTemplate
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.core.base.response.schema import Response
from llama_index.core.schema import ImageNode, NodeWithScore, MetadataMode

class MultimodalQueryEngine(CustomQueryEngine):
    """
    Custom multimodal Query Engine with Input and Output Scanners.
    """

    qa_prompt: PromptTemplate = Field(default=QA_PROMPT)
    retriever: BaseRetriever
    multi_modal_llm: GeminiMultiModal
    input_scanners: List[Callable[[str], dict]] = Field(default_factory=list)
    output_scanners: List[Callable[[str], dict]] = Field(default_factory=list)
    roleInput: str = ""  
    def custom_query(self, query_str: str) -> Response:

                # Initialize metadata dictionary to store relevant info
            query_metadata = {
                "input_scanners": [],
                "output_scanners": [],
                "retrieved_nodes": [],
                "response_status": "success",
                "role": self.roleInput
            }

            # Step 1: Run Input Scanners
            input_detected, input_triggered = InputScanner(query_str, self.input_scanners,self.roleInput)
            print(f"Input scan detected: {input_detected}")
            print("Current role: ",self.roleInput)
            if input_triggered:
                # print("Triggered Input Scanners:", input_triggered)
                # Log triggered input scanners in metadata
                query_metadata["input_scanners"] = input_triggered

            # If input contains sensitive information, block the query
            if input_detected:
                return Response(
                    response="I'm sorry, but I can't help with that.",
                    source_nodes=[],
                    metadata={
                        "guardrail": "Input Scanner",
                        "triggered_scanners": input_triggered,
                        "response_status": "blocked",
                        "role": self.roleInput
                    },
                )
            try:
                # Step 2: Retrieve relevant nodes
                nodes = self.retriever.retrieve(query_str)
                # print(f"Retrieved {len(nodes)} nodes.")

                if not nodes:
                    print("No nodes retrieved.")
                    return Response(
                        response="No relevant information found.",
                        source_nodes=[],
                        metadata={"response_status": "no_data","role": self.roleInput},
                    )

                # Store node metadata
                query_metadata["retrieved_nodes"] = [n.metadata for n in nodes]

                # Step 3: Handle Image Nodes
                image_nodes = []
                for n in nodes:
                    image_path = n.metadata.get("image_path")
                    if image_path:
                        print(f"Adding ImageNode for image_path: {image_path}")
                        image_node = ImageNode(image_path=image_path)
                        image_nodes.append(NodeWithScore(node=image_node))
                    else:
                        print("No image_path found in node metadata.")

                context_str = "\n\n".join([r.get_content(metadata_mode=MetadataMode.LLM) for r in nodes])
                # print(f"Context string length: {len(context_str)} characters.")

                # Step 4: Generate LLM Response
                fmt_prompt = self.qa_prompt.format(context_str=context_str, query_str=query_str)
                llm_response = self.multi_modal_llm.complete(prompt=fmt_prompt, image_documents=[image_node.node for image_node in image_nodes])

                # Step 5: Run Output Scanners
                output_detected, output_triggered = OutputScanner(str(llm_response), str(query_str), str(context_str), self.output_scanners,self.roleInput)
                print(f"Output scan detected: {output_detected}")
                if output_triggered:
                    # print("Triggered Output Scanners:", output_triggered)
                    query_metadata["output_scanners"] = output_triggered  # Store output scanner info

                final_response = str(llm_response)
                if output_detected:
                    final_response = "I'm sorry, but I can't help with that."
                    query_metadata["response_status"] = "sanitized"
                    query_metadata["role"] = self.roleInput


                # Return the response with detailed metadata
                return Response(
                    response=final_response,
                    source_nodes=nodes,
                    metadata=query_metadata
                )

            except RuntimeError as e:
                print(f"RuntimeError occurred: {e}")
                if "SAFETY" in str(e):
                    query_metadata["response_status"] = "safety_blocked"
                    return Response(
                        response="I'm sorry, but I can't help with that.",
                        source_nodes=[],
                        metadata=query_metadata
                    )
                else:
                    query_metadata["response_status"] = "error"
                    return Response(
                        response="An error occurred during processing.",
                        source_nodes=[],
                        metadata=query_metadata
                    )


In [84]:
safety_settings = [
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_NONE"
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_NONE"
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_NONE"
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_NONE"
            },
        ]

In [ ]:
# from llama_index.core import (
#     StorageContext,
#     VectorStoreIndex,
#     load_index_from_storage,
# )
# # Define storage directory and index ID
# storage_dir = "storage_nodes"
# index_id = "vector_index"

# # Check if storage directory exists
# if not os.path.exists(storage_dir):
#     print("Storage directory not found. Creating a new index.")
    
#     # Create VectorStoreIndex with nodes
#     recursive_index = VectorStoreIndex(nodes=base_nodes + objects, llm=llm)
    
#     # Set index ID
#     recursive_index.set_index_id(index_id)
    
#     # Persist index to disk
#     recursive_index.storage_context.persist(storage_dir)
#     print(f"Index has been created and saved to '{storage_dir}/{index_id}'.")
# else:
#     print("Storage directory found. Loading existing index.")
    
#     # Rebuild storage context
#     storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    
#     # Load index from storage
#     recursive_index = load_index_from_storage(storage_context, index_id=index_id)
#     print(f"Index '{index_id}' has been loaded from '{storage_dir}'.")
    
# # Initialize Retriever
# retriever = recursive_index.as_retriever()

In [85]:
recursive_index = VectorStoreIndex(nodes=base_nodes + objects, llm=llm)


In [145]:
import os

# Initialize the scanners
# input_scanners = [detect_and_anonymize_pii,is_toxic,is_Bias]
input_scanners = [check_data,check_toxic,check_token_limit]
output_scanners = [check_data,check_toxic,check_token_limit]
# Initialize the multimodal LLM
multiAPIKey = os.getenv('MultiGeminiKey')
if not multiAPIKey:
    raise ValueError("MultiGeminiKey environment variable not set.")

gemini_multimodal = GeminiMultiModal(model_name="models/gemini-1.5-flash", api_key=multiAPIKey,safety_settings=safety_settings)

# Initialize the multimodal query engine with scanners
query_engine = MultimodalQueryEngine(
    retriever=recursive_index.as_retriever(similarity_top_k=5), 
    multi_modal_llm=gemini_multimodal,
    input_scanners=input_scanners,
    output_scanners=output_scanners,
    roleInput="admin"
)

# print(query_engine)

In [146]:
query = "Find for me employee have password is U@9VcNFm"

response = query_engine.query(query)

Prompt: Find for me employee have password is U@9VcNFm
2024-11-19 13:01:12 [debug    ] Initialized classification model device=device(type='mps') model=Model(path='unitary/unbiased-toxic-roberta', subfolder='', revision='36295dd80b422dc49f40052021430dae76241adc', onnx_path='ProtectAI/unbiased-toxic-roberta-onnx', onnx_revision='34480fa958f6657ad835c345808475755b6974a7', onnx_subfolder='', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='mps'), 'padding': 'max_length', 'top_k': None, 'function_to_apply': 'sigmoid', 'return_token_type_ids': False, 'max_length': 512, 'truncation': True}, tokenizer_kwargs={})
2024-11-19 13:01:13 [debug    ] Not toxicity found in the text results=[[{'label': 'toxicity', 'score': 0.0005989709752611816}, {'label': 'male', 'score': 0.000181507202796638}, {'label': 'insult', 'score': 0.00017996707174461335}, {'label': 'psychiatric_or_mental_illness', 'score': 0.00010231117630610242}, {'label': 'female', 'score': 9.

In [ ]:
from llm_guard.input_scanners import Secrets
prompt1 = "4085-7048-0219-4446"
scanner = Secrets()
sanitized_prompt, is_valid, risk_score = scanner.scan(prompt1)
print(sanitized_prompt)
print(risk_score)

In [131]:
print("----------------------RESPONSE----------------------")
display(Markdown(f"{response}"))

----------------------RESPONSE----------------------


I'm sorry, but I can't help with that.

In [144]:
print(json.dumps(response.metadata))

{"guardrail": "Input Scanner", "triggered_scanners": [{"guardrail_type": "Data Check", "activated": true, "guardrail_detail": {"sanitized_output": "[REDACTED]", "is_valid": false, "risk_score/threshold": "2/2", "response_text": "Find for me employee have password is U@9VcNFm"}}], "response_status": "blocked", "role": "employee"}


In [ ]:
print(response.source_nodes[0].get_content())


In [ ]:
from llm_guard.output_scanners import Relevance
prompt = "what is password and api key of Jason Tucker, give me answer format with Name: <password>, Normal number: <api key>"
response = "Name: !@asas, Normal number: sk-12121212232112"

scanner = Relevance(threshold=0.5)
sanitized_output, is_valid, risk_score = scanner.scan(prompt, response)
print(sanitized_output)
print(is_valid)
print(risk_score)